Load a dataset with clean EEG data, add noise and save the noisy dataset

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch 
from contrib.eeg import data, utils_eeg

from pathlib import Path
import json
from scipy.io import loadmat, savemat
import os

In [84]:
## parameters to load the data
datafolder = "/home/reynaudsarah/Documents/Data/simulation"
head_model = {
    "electrode_montage": "standard_1020", 
    "source_sampling": "ico3", 
    "subject_name": "fsaverage", 
    "orientation": "constrained"
}

simu_name = "_dl_ses_fsaverage_st1020_visu_"
# simu_name = "ses_125ms_test"
to_load = 1000
config_file = Path(
    datafolder,
    head_model['subject_name'], head_model['orientation'], head_model['electrode_montage'], head_model['source_sampling'], "simu",
    simu_name, f"{simu_name}{head_model['source_sampling']}_config.json"
            )
print(config_file)
with open(config_file) as f : 
    config_dict = json.load(f)
print(config_dict.keys())

/home/reynaudsarah/Documents/Data/simulation/fsaverage/constrained/standard_1020/ico3/simu/_dl_ses_fsaverage_st1020_visu_/_dl_ses_fsaverage_st1020_visu_ico3_config.json
dict_keys(['electrode_space', 'source_space', 'rec_info', 'ids'])


In [ ]:
simu_path = Path(  
    datafolder, head_model['subject_name'], head_model['orientation'], head_model['electrode_montage'] ,head_model['source_sampling'], "simu"
    )
print(simu_path)
info_file = Path( simu_path, simu_name ,f"{simu_name}{head_model['source_sampling']}_match_json_file.json" )
with open(info_file) as f : 
    match_info_dict = json.load(f)

data_ids = np.array( 
    list( match_info_dict.keys() )
    )
print(match_info_dict['id_1'].keys())
n_samples = len(data_ids)

/home/reynaudsarah/Documents/Data/simulation/fsaverage/constrained/standard_1020/ico3/simu
dict_keys(['act_src_file_name', 'noise_src_file_name', 'eeg_file_name', 'md_json_file_name'])


In [88]:
snr = 5

In [89]:
simu_path

PosixPath('/home/reynaudsarah/Documents/Data/simulation/fsaverage/constrained/standard_1020/ico3/simu')

In [90]:
n_samples

100

In [91]:
for i in range(n_samples): 
    # load
    path_eeg = utils_eeg.replace_root_fn( simu_path, simu_name, match_info_dict[data_ids[i]]['eeg_file_name'])
    eeg = utils_eeg.load_mat( str(path_eeg) )['eeg_data']['EEG']
    # add noise to SNR
    inf = np.min(eeg)
    sup = np.max(eeg)
    if snr < 50:
        eeg_noise = utils_eeg.array_range_scaling(
                utils_eeg.add_noise_snr( snr,eeg ), inf, sup
            ) 
    # visu to check range of data remains the same
    # plt.figure()
    # plt.subplot(121)
    # for e in range(eeg.shape[0]): 
    #     plt.plot(eeg[e,:])
    # plt.subplot(122)
    # for e in range(eeg.shape[0]): 
    #     plt.plot(eeg_noise[e,:])

    # save file
    to_save = {
        "eeg_data": {
            "EEG": eeg_noise
        }
    }
    save_path = Path( simu_path, simu_name, 'eeg', f'{snr}db' )
    os.makedirs(save_path, exist_ok=True)
    save_file = Path( save_path, f"{data_ids[i].split('_')[-1]}_eeg.mat")
    savemat(save_file, to_save)

In [92]:
match_info_dict[data_ids[5]]

{'act_src_file_name': '/home/reynaudsarah/Documents/Data/simulation/constrained/standard_1020/ico3/simu/_dl_ses_fsaverage_st1020_visu_/sources/Jact/6_src_act.mat',
 'noise_src_file_name': '/home/reynaudsarah/Documents/Data/simulation/constrained/standard_1020/ico3/simu/_dl_ses_fsaverage_st1020_visu_/sources/Jnoise/6_src_noise.mat',
 'eeg_file_name': '/home/reynaudsarah/Documents/Data/simulation/constrained/standard_1020/ico3/simu/_dl_ses_fsaverage_st1020_visu_/eeg/infdb/6_eeg.mat',
 'md_json_file_name': '/home/reynaudsarah/Documents/Data/simulation/constrained/standard_1020/ico3/simu/_dl_ses_fsaverage_st1020_visu_/md/6_md_json_flie.json'}

In [93]:
# chemin: 
# datafolder, subject_name, orientation, electrode_montage, source_sampling, "simu", simu_name, "eeg", f"{snr}db", f"{data_ids[i].split('_')[-1]}_eeg.mat"